In [48]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()
sc =SparkContext()
sqlCtx = SQLContext(sc)


In [49]:
# Campaign details from MARK 
rdd = sc.textFile("gs://ds-url-catag/campaign_analysis/campaign_details.csv") \
    .map(lambda line: line.split(";"))     
first_line = rdd.first()
campaigns_list = rdd.filter(lambda line: line != first_line).toDF(['orders_id','order','audseg_idname','audseg_id','impr','click','category'])


In [50]:
#details of the campaigns - This has to be removed 
campaign_details = sqlCtx.read.parquet("gs://ds-url-catag/campaign_analysis/processed_orders/")

#tagging the campagin list - the campaings that use our model
campaigns_list = campaigns_list.withColumn("To_check",lit(True))

#get the details of the campagins list
cond = [campaign_details.order_id == campaigns_list.orders_id,campaign_details.target_audienceid == campaigns_list.audseg_id]

#getting the detailds of all the campaign list
campaignslisttier1 = campaigns_list.join(campaign_details,cond,'left')

In [51]:
#reading all the campaign details obtained from datapreprocessing and scraping - previous two steps 
filtered_campaigns = sqlCtx.read.parquet("gs://ds-url-catag/campaign_analysis/campaign_details/")
filtered_campaigns = filtered_campaigns.select(col("order_id").alias("orderid"),col("Order_name"),col("IABTier1Categorization"),col("IOEndDate"),col("IOStartDate"))

#get the details of the filtered campaigns
cond = [campaign_details.order_id == filtered_campaigns.orderid]

#getting the details of all the campaign list
filtcamptier1 = filtered_campaigns.join(campaign_details,cond,'left')

#To filter the campaigns before applying model "TO_check !=  True"
filtcamptier2 = filtcamptier1.join(campaigns_list,filtcamptier1.orderid == campaigns_list.orders_id,'left')
filtcamptier2 =filtcamptier2.withColumn("To_check",when(col('To_check') == True,col('To_check')).otherwise(False))
filtcamptier3 = filtcamptier2.where(col('To_check') == False )

In [52]:
#comparing campaigns 
from pyspark.sql.types import BooleanType,StringType
import re
#'DBS',indosat','grab','Google roadblock','uber','opera']

#GOJEK
def regex_filter(x):
    list_campaign = ['Gojek']
    filter_status = ''
    string_x = x[:5]
    for camp in list_campaign:
        if camp.lower() in string_x.lower():
            filter_status = True
    return filter_status
filter_udf = udf(regex_filter, BooleanType())

#Before application of the model
filtcamptier3 = filtcamptier3.withColumn('Filter_campaigns',filter_udf(filtcamptier3.Order_name))    

In [53]:
#gojek old
go_jek_old = filtcamptier3.where(col('Filter_campaigns') == True)
go_jek_old = go_jek_old.withColumn("filter",when(col('Age_group_5') != "null",True).otherwise(False))
go_jek_final = go_jek_old.where(col('filter') == False)
# go_jek_final = go_jek_final.withColumn("CTR",((col('clicks')/col('impressions'))*100))

In [54]:
#gojek new
campaignslisttier1 = campaignslisttier1.withColumn('Filter_campaigns',filter_udf(campaignslisttier1.order))
go_jek_new = campaignslisttier1.where(col('Filter_campaigns') == True)


In [106]:
# #calculating statistics
# # #no of campaigns
# # print go_jek_final.count()

# #calculate the difference between timestamps
# from pyspark.sql import functions as F
# timeFmt = "yyyy-MM-dd"
# timeDiff = (F.unix_timestamp('IOEndDate', format=timeFmt)
#             - F.unix_timestamp('IOStartDate', format=timeFmt))
# #for old
# go_jek_final = go_jek_final.withColumn("Duration", (((timeDiff/60)/60/24)))

# #avg no of days 
# print go_jek_final.agg(avg(col('Duration'))).show()

# # impressions and clicks 
# go_jek_final = go_jek_final.withColumn("CTR",((col('clicks')/col('impressions'))*100))

# #CTR calculation - check the conditions to filter 
# rem_out = go_jek_final.where(col('CTR').isNotNull())
# print rem_out.agg(avg(col('CTR'))).show()


# #Average Budget 
# print go_jek_final.agg(avg(col('budget'))).show(1,False)

print go_jek_final.groupby('target_country').count().show()

+---------------+-----+
| target_country|count|
+---------------+-----+
|        Jakarta|    2|
|          Bogor|    2|
|          Depok|    2|
|      Palembang|    2|
|         Bekasi|    2|
|      Indonesia|   10|
|       Surabaya|    1|
|          Medan|    1|
|        Bandung|    1|
|South Tangerang|    2|
+---------------+-----+

None


In [108]:
# #no of campaigns
# print go_jek_new.count()

# #calculate the difference between timestamps
# from pyspark.sql import functions as F
# timeFmt = "yyyy-MM-dd"
# timeDiff = (F.unix_timestamp('End_date', format=timeFmt)
#             - F.unix_timestamp('start_date', format=timeFmt))
# #for old
# go_jek_new = go_jek_new.withColumn("Duration", (((timeDiff/60)/60/24)))

# #avg no of days 
# print go_jek_new.agg(avg(col('Duration'))).show()

# # impressions and clicks 
# go_jek_new = go_jek_new.withColumn("CTR",((col('clicks')/col('impressions'))*100))

# #CTR calculation - check the conditions to filter 
# rem_out_new = go_jek_new.where(col('CTR') !=0)
# print rem_out_new.agg(avg(col('CTR'))).show()

# #Average Budget 
# print go_jek_new.agg(avg(col('budget'))).show(1,False)


print go_jek_new.groupby('target_country').count().show()

+---------------+-----+
| target_country|count|
+---------------+-----+
|      Tangerang|    1|
|       Makassar|    4|
|        Jakarta|   20|
|           null|    2|
|          Bogor|   20|
|          Depok|   20|
|         Bekasi|   20|
|      Palembang|    4|
|       Surabaya|    4|
|          Medan|    4|
|        Bandung|    4|
|         Banten|   10|
|South Tangerang|    9|
+---------------+-----+

None


+--------------------+
|avg(budget)         |
+--------------------+
|4.064284397644192E12|
+--------------------+

None


In [109]:
go_jek_final.select(['orderid','Order_name','Duration','CTR','budget','currency_code','ad_type_1','Age_group_1','Age_group_2','target_country','IABTier1Categorization']).show(44)

+----------+--------------------+--------+------------------+--------------+-------------+---------+-----------+-----------+---------------+----------------------+
|   orderid|          Order_name|Duration|               CTR|        budget|currency_code|ad_type_1|Age_group_1|Age_group_2| target_country|IABTier1Categorization|
+----------+--------------------+--------+------------------+--------------+-------------+---------+-----------+-----------+---------------+----------------------+
|2126562946|Gojek-Suply-Alt-2...|    31.0|              null|        100000|          IDR|     dnau|      35-44|      13-17|      Indonesia|            Automotive|
|2126562946|Gojek-Suply-Alt-2...|    31.0|              null|        100000|          IDR|     dnau|      35-44|      18-24|      Indonesia|            Automotive|
|2126562946|Gojek-Suply-Alt-2...|    31.0|              null|        100000|          IDR|     dnau|      35-44|      18-24|      Indonesia|            Automotive|
|2177837480|Goje

In [ ]:
go_jek_new.agg(avg(col('Duration'))).show()


In [111]:
# go_jek_new.select(['orders_id','order','clicks','Duration','CTR','budget','currency_code','ad_type_1','Age_group_1','Age_group_2','Age_group_3','Age_group_4','target_country','IABTier1Categorization']).show(123)

In [110]:
go_jek_new.columns

['orders_id',
 'order',
 'audseg_idname',
 'audseg_id',
 'impr',
 'click',
 'category',
 'To_check',
 'line_item_id',
 'order_id',
 'line_item_name',
 'budget',
 'currency_code',
 'impressions',
 'clicks',
 'target_country',
 'target_os',
 'start_date',
 'End_date',
 'Gender',
 'ad_type_1',
 'ad_type_2',
 'ad_type_3',
 'ad_type_4',
 'Age_group_1',
 'Age_group_2',
 'Age_group_3',
 'Age_group_4',
 'Age_group_5',
 'Age_group_6',
 'Age_group_7',
 'Age_group_8',
 'channels_1',
 'channels_2',
 'target_audienceid',
 'Filter_campaigns',
 'Duration',
 'CTR']